In [ ]:
import numpy as np
import pandas as pd

nrows_to_load = 10000000

unique_vals = {
    'category_1': ['N', 'Y', 'Z'],
    'category_3': ['A', 'B', 'C', 'Z'],
    'category_2': [1.0, -1, 3.0, 5.0, 2.0, 4.0],
    'state_id': [16,9,-1,11,15,17,5,24,19,23,3,8,18,7,4,22,13,1,10,21,20,14,2,12,6],
    'category_4': ['N', 'Y', 'Z'],
    'most_recent_purchases_range': ['E', 'D', 'C', 'B', 'A', 'Z'],
    'most_recent_sales_range': ['E', 'D', 'C', 'B', 'A', 'Z']
}

# Load in all historical transactions.
hist_df = pd.concat([
    pd.read_csv(
        "../input/historical_transactions.csv", 
        index_col='merchant_id'
    ),
    pd.read_csv(
        "../input/new_merchant_transactions.csv", 
        index_col='merchant_id'
    )
])

In [3]:
# Load in merchant information.
merchants_df = (
    pd.read_csv("../input/merchants.csv")
    .drop_duplicates(subset='merchant_id', keep='first')
    .set_index('merchant_id')
)

In [ ]:
def process_batch(hist_df, merchants_df, i, final_set=False):
    if final_set:
        hist_df_batch = hist_df.iloc[i*nrows_to_load:, :]
    else:
        hist_df_batch = hist_df.iloc[i*nrows_to_load:(i+1)*nrows_to_load, :]
    print("Processing batch: ", i)
    print("Batch size: ", hist_df_batch.shape)

    cols_to_use = merchants_df.columns.difference(hist_df_batch.columns)

    # Join historical transactions with merchant information where 
    # transactions were made and drop uneccessary columns right after that.
    hist_transactions = (
        hist_df_batch
        .join(
            merchants_df[cols_to_use],
            how='left',
            on='merchant_id',
        )
        .reset_index()
        .drop(columns=[
            'avg_sales_lag12', 'avg_sales_lag6', 'avg_sales_lag3', 
            'active_months_lag3', 'active_months_lag6', 
            'avg_purchases_lag3', 'avg_purchases_lag6', 
            'merchant_group_id'
        ])
    )

    # Create a column for number of days between purchases using a window function.
    hist_transactions['purchase_date'] = pd.to_datetime(hist_transactions['purchase_date'])
    hist_transactions['purchase_date_day'] = pd.to_datetime(hist_transactions['purchase_date']).dt.day
    hist_transactions['days_between_purch'] = hist_transactions['purchase_date_day'].rolling(2).sum()

    # Some info is missing because not all transactions have a 
    # merchant matched with them. We don't want to lose transactions because 
    # then we might not have any data for some card_ids, so will fill 
    # out missing rows with special values (-1 for numerical and 'Z' for categorical).
    hist_transactions['days_between_purch'] = hist_transactions['days_between_purch'].fillna(30) # 30 - is the average value.
    hist_transactions['category_2'] = hist_transactions['category_2'].fillna(-1)
    hist_transactions['category_3'] = hist_transactions['category_3'].fillna('Z')
    hist_transactions['category_4'] = hist_transactions['category_4'].fillna('Z')
    hist_transactions['active_months_lag12'] = hist_transactions['active_months_lag12'].fillna(-1)
    hist_transactions['avg_purchases_lag12'] = hist_transactions['avg_purchases_lag12'].fillna(-1)
    hist_transactions['most_recent_purchases_range'] = hist_transactions['most_recent_purchases_range'].fillna('Z')
    hist_transactions['most_recent_sales_range'] = hist_transactions['most_recent_sales_range'].fillna('Z')
    hist_transactions['numerical_1'] = hist_transactions['numerical_1'].fillna(hist_transactions['numerical_1'].mean())
    hist_transactions['numerical_2'] = hist_transactions['numerical_2'].fillna(hist_transactions['numerical_2'].mean())
    hist_transactions['authorized_flag_binary'] = hist_transactions['authorized_flag'].apply(lambda x: 1 if x == 'Y' else 0)

    categorical_cols = [
        'category_1', 'category_3', 'category_2', 'category_4','state_id',
        'most_recent_purchases_range', 'most_recent_sales_range'
    ]
    
    # Since we are processing data in batches, some batches might not span all
    # of the possible categories within categorical columns, so we are going
    # to preset all categorical columns with all possible categories that 
    # they can take on, even if some categories are missing in the batch of 
    # data that is currently being processed.
    for cat in categorical_cols:
        hist_transactions[cat] = hist_transactions[cat].astype('category', categories=unique_vals[cat])

    to_process_cols = [
        'category_1', 'category_2', 'category_3', 'category_4', 'state_id', 
        'most_recent_purchases_range', 'most_recent_sales_range',
    ]
    vect_category_cols = []

    # Convert all categorical columns into dummy variables.
    for cat in to_process_cols:
        vect_category_cols += [cat + '_' + str(col) for col in list(hist_transactions[cat].unique())]
        dummies = pd.get_dummies(hist_transactions[cat], prefix=cat)
        hist_transactions = pd.concat([hist_transactions, dummies], axis=1)

    def process_func(x):
        d = {}
        
        d['avg_month_lag'] = x['month_lag'].mean()
        d['avg_installments'] = x['installments'].mean()
        d['avg_days_between_purch'] = x['days_between_purch'].mean()
        d['std_days_between_purch'] = x['days_between_purch'].std()
        d['num_authorized'] = x['authorized_flag_binary'].sum()
        d['avg_purchase_amount'] = x['purchase_amount'].mean()
        d['std_purchase_amount'] = x['purchase_amount'].std()
        d['min_purchase_amount'] = x['purchase_amount'].min()
        d['max_purchase_amount'] = x['purchase_amount'].max()
        d['num_purchases'] = x['purchase_amount'].count()
        d['num_unique_merchants'] = x['merchant_id'].count()
        d['avg_numerical_1'] = x['numerical_1'].mean()
        d['std_numerical_1'] = x['numerical_1'].std()
        d['min_numerical_1'] = x['numerical_1'].min()
        d['max_numerical_1'] = x['numerical_1'].max() 
        d['avg_numerical_2'] = x['numerical_2'].mean()
        d['std_numerical_2'] = x['numerical_2'].std()
        d['min_numerical_2'] = x['numerical_2'].min()
        d['max_numerical_2'] = x['numerical_2'].max()
        d['avg_active_months_lag12'] = x['active_months_lag12'].mean()
        d['avg_purchases_lag12'] = x['avg_purchases_lag12'].mean()

        for col in vect_category_cols:
            d[col] = x[col].sum()

        return pd.Series(d, index=[
            'avg_month_lag', 'avg_installments', 'avg_days_between_purch', 'std_days_between_purch',
            'num_authorized', 'avg_purchase_amount', 'std_purchase_amount',
            'min_purchase_amount', 'max_purchase_amount', 'num_purchases', 'avg_numerical_1', 'std_numerical_1',
            'min_numerical_1', 'max_numerical_1', 'avg_numerical_2', 'std_numerical_2',
            'min_numerical_2', 'max_numerical_2', 'avg_active_months_lag12', 'avg_purchases_lag12', 'num_unique_merchants'
        ] + vect_category_cols)

    # Since historical transactions can have more than one transaction made by 
    # the same card id, we group the dataset by card id and create summary 
    # statistics to describe transactions made by that card is.
    final_df = hist_transactions.groupby(by='card_id').apply(process_func)

    # Save the processed batch into a csv file.
    print('Saving processed DF:', final_df.shape)
    print("===============================")
    final_df.to_csv('df_new_' + str(i) + '.csv', sep=',')

In [ ]:
# Find the ideal number of batches to break dataset into.
num_slices = int(hist_df.shape[0] / nrows_to_load) + 1

# For each batch, run preprocessing function and save result to csv files.
for i in range(num_slices):
    process_batch(hist_df, merchants_df, i, final_set=True if (i+1)*nrows_to_load >= hist_df.shape[0] else False)